# <center><font color='blue'>SENTIMENT ANALYSIS: COVID</center></font>

## Table of contents
- [1 - Objetivos](#1)
- [2 - Librerías necesarias](#2)
- [3 - Carga y visualización de datos](#3)
- [4 - Pre-procesamiento de datos](#4)
    - [4.1. - Datos faltantes](#4.1)
    - [4.2. - Data Categóricos](#4.2)
    - [4.3. - Balanceo de clases](#4.3)
    - [4.4. - Pre-Procesamiento especial para NLP](#4.4)
- [5 - Modelos](#5)
    - [5.1. - Modelo 1](#5.1)
    - [5.2. - Modelo 2](#5.2)
    - [5.3. - Modelo 3](#5.3)
    - [5.4. - Modelo 4](#5.4)
    - [5.4. - Modelo 5](#5.5)
    - [5.4. - Modelo 6](#5.6)
    - [5.4. - Modelo 7](#5.7)
- [6 - Comparando los modelos y eligiendo el mejor](#6)
- [7 - Ajuste de hiperparámetros](#7)
- [8 - Predicciones con el modelo final](#8)
- [9 - Guardando el modelo](#9)
- [10 - Conclusiones](#10)

<a name="1"></a>
## <b> <font color='blue'> 1. Objectives </font> </b>

Practice with a natural language processing problem.
<br>
Here, given a set of tweets, analyze whether the sentiment is positive, negative, or neutral.

<a name="2"></a>
## <b> <font color='blue'> 2. Setup </font> </b>

In [1]:
# do not print info and warning messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [2]:
# basic imports
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
import pandas as pd

2024-07-10 20:50:00.056418: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-10 20:50:00.056468: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-10 20:50:00.057664: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


<a name="3"></a>
## <b> <font color='blue'> 3. Data loading and visualization </font> </b>

In [3]:
!ls data

Corona_NLP_test.csv  Corona_NLP_train.csv


We have 2 datasets, one for training and one for test.

In [4]:
train_data = pd.read_csv('data/Corona_NLP_train.csv',encoding='latin-1')
test_data = pd.read_csv('data/Corona_NLP_test.csv',encoding='latin-1')

In [5]:
# let's see some of the training data
pd.set_option('display.max_colwidth', None) # que No recorte el texto

train_data.head(10)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order,Positive
2,3801,48753,Vagabonds,16-03-2020,"Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P",Positive
3,3802,48754,NaN,16-03-2020,"My food stock is not the only one which is empty...\r\r\n\r\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\r\nStay calm, stay safe.\r\r\n\r\r\n#COVID19france #COVID_19 #COVID19 #coronavirus #confinement #Confinementotal #ConfinementGeneral https://t.co/zrlG0Z520j",Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COVID19 outbreak.\r\r\n\r\r\nNot because I'm paranoid, but because my food stock is litteraly empty. The #coronavirus is a serious thing, but please, don't panic. It causes shortage...\r\r\n\r\r\n#CoronavirusFrance #restezchezvous #StayAtHome #confinement https://t.co/usmuaLq72n",Extremely Negative
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,"As news of the regionÂs first confirmed COVID-19 case came out of Sullivan County last week, people flocked to area stores to purchase cleaning supplies, hand sanitizer, food, toilet paper and other goods, @Tim_Dodson reports https://t.co/cfXch7a2lU",Positive
6,3805,48757,"35.926541,-78.753267",16-03-2020,"Cashier at grocery store was sharing his insights on #Covid_19 To prove his credibility he commented ""I'm in Civics class so I know what I'm talking about"". https://t.co/ieFDNeHgDO",Positive
7,3806,48758,Austria,16-03-2020,Was at the supermarket today. Didn't buy toilet paper. #Rebel\r\r\n\r\r\n#toiletpapercrisis #covid_19 https://t.co/eVXkQLIdAZ,Neutral
8,3807,48759,"Atlanta, GA USA",16-03-2020,"Due to COVID-19 our retail store and classroom in Atlanta will not be open for walk-in business or classes for the next two weeks, beginning Monday, March 16. We will continue to process online and phone orders as normal! Thank you for your understanding! https://t.co/kw91zJ5O5i",Positive
9,3808,48760,"BHAVNAGAR,GUJRAT",16-03-2020,"For corona prevention,we should stop to buy things with the cash and should use online payment methods because corona can spread through the notes. Also we should prefer online shopping from our home. It's time to fight against COVID 19?. #govindia #IndiaFightsCorona",Negative


In [6]:
print(f" Training data: (rows, cols) = {train_data.shape}")
print(f" Training data type:  {type(train_data)}")
print(f" Test data: (rows, cols) = {test_data.shape}")
print(f" Test data type:  {type(test_data)}")

 Training data: (rows, cols) = (41157, 6)
 Training data type:  <class 'pandas.core.frame.DataFrame'>
 Test data: (rows, cols) = (3798, 6)
 Test data type:  <class 'pandas.core.frame.DataFrame'>


<a name="4"></a>
## <b> <font color='blue'> 4. Data Pre-processing </font> </b>


<a name="4.1"></a>
### <b> <font color='#5499C7'> 4.1. Missing data </font> </b>

In [7]:
print(f'Missing data train:\n {train_data.isnull().sum()} \n')
print(f'Missing data test:\n {test_data.isnull().sum()}')

Missing data train:
 UserName            0
ScreenName          0
Location         8590
TweetAt             0
OriginalTweet       0
Sentiment           0
dtype: int64 

Missing data test:
 UserName           0
ScreenName         0
Location         834
TweetAt            0
OriginalTweet      0
Sentiment          0
dtype: int64


There are no missing data in the columns we care about (Original Tweet and Sentiment).

<a name="4.2"></a>
### <b> <font color='#5499C7'> 4.2. Feature Extraction </font> </b>

Our features will be the column "OriginalTweet" and our target the column "Sentiment"


In [8]:
X_train = train_data['OriginalTweet']
y_train = train_data['Sentiment']
X_test = test_data['OriginalTweet']
y_test = test_data['Sentiment']

In [9]:
type(X_train)

pandas.core.series.Series

<a name="4.3"></a>
### <b> <font color='#5499C7'> 4.3. Categorical data </font> </b>

Our target is categorical, lets see the different options:

In [10]:
y_train.unique()

array(['Neutral', 'Positive', 'Extremely Negative', 'Negative',
       'Extremely Positive'], dtype=object)

Lets see if there are other options in the test set:

In [11]:
y_test.unique()

array(['Extremely Negative', 'Positive', 'Extremely Positive', 'Negative',
       'Neutral'], dtype=object)

In [12]:
are_same = np.array_equal(np.sort(y_train.unique()), np.sort(y_test.unique()))
print(f"Are unique values the same? {are_same}")

Are unique values the same? True


We are not so interested in whether a sentiment is positive or extremely positive, therefore we will rename the label extremely positive to positive.

In [13]:
y_train = y_train.replace('Extremely Positive', 'Positive').replace('Extremely Negative', 'Negative')
y_test = y_test.replace('Extremely Positive', 'Positive').replace('Extremely Negative', 'Negative')

Check

In [14]:
print(y_train.unique(), y_test.unique())

['Neutral' 'Positive' 'Negative'] ['Negative' 'Positive' 'Neutral']


We will not consider that there is an ordinal relationship, therefore we will encode the labels using one-hot encoding.

Remember that we need to apply the same encoding to train and test.

In [15]:
from sklearn.preprocessing import OneHotEncoder


# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

# Fit the encoder on the train set
encoder.fit(y_train.values.reshape(-1, 1))

# Transform both train and test sets
one_hot_train = encoder.transform(y_train.values.reshape(-1, 1))
one_hot_test = encoder.transform(y_test.values.reshape(-1, 1))

# Convert to DataFrame for easier inspection
y_train = pd.DataFrame(one_hot_train, columns = encoder.get_feature_names_out(['sentiment']))
y_test = pd.DataFrame(one_hot_test, columns = encoder.get_feature_names_out(['sentiment']))

print("One-hot encoded train set:")
print(y_train)
print("\nOne-hot encoded test set:")
print(y_test)


One-hot encoded train set:
       sentiment_Negative  sentiment_Neutral  sentiment_Positive
0                     0.0                1.0                 0.0
1                     0.0                0.0                 1.0
2                     0.0                0.0                 1.0
3                     0.0                0.0                 1.0
4                     1.0                0.0                 0.0
...                   ...                ...                 ...
41152                 0.0                1.0                 0.0
41153                 1.0                0.0                 0.0
41154                 0.0                0.0                 1.0
41155                 0.0                1.0                 0.0
41156                 1.0                0.0                 0.0

[41157 rows x 3 columns]

One-hot encoded test set:
      sentiment_Negative  sentiment_Neutral  sentiment_Positive
0                    1.0                0.0                 0.0
1           

In this example, handle_unknown='ignore' is used to ensure that any categories present in the test set but not in the train set are handled gracefully. This is useful when you have categories in the test set that were not seen during training.

<a name="4.4"></a>
### <b> <font color='#5499C7'> 4.4. Class balance </font> </b>

Let's see if the classes are balanced.


-> arriba de la codificación!!!!!!!!!!


In [16]:
train_data['Sentiment'].value_counts()

Positive              11422
Negative               9917
Neutral                7713
Extremely Positive     6624
Extremely Negative     5481
Name: Sentiment, dtype: int64

<a name="4.5"></a>
### <b> <font color='#5499C7'> 4.5. Special pre-processing for NLP </font> </b>

Let's preprocess the text from OriginalTweet, for which:

- We will remove stop words.
- We will remove certain special characters, like "@".
- We will apply lemmatization.


<b>Note:</b> We will also remove punctuation, convert everything to lowercase, and tokenize later using TextVectorization.

We will download and print to see the stop words.

In [17]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/marcos/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/marcos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/marcos/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

We will remove the stop words and apply lemmatization:

In [21]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Inicializar lematizador
lemmatizer = WordNetLemmatizer()

# stop words
stop_words = set(stopwords.words('english'))

# Función para quitar palabras de parada y lematizar un texto
def preprocess_text(text):
    words = word_tokenize(text)
    filtered_words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Aplicar la función a la columna 'OriginalTweet' del dataset, tanto en train como test
X_train = X_train.apply(preprocess_text)
X_test = X_test.apply(preprocess_text)

<a name="4.6"></a>
### <b> <font color='#5499C7'> 4.6. Train/validation split </font> </b>

In [22]:
from sklearn.model_selection import train_test_split 

# Dividir los datos en conjuntos de entrenamiento y validación
#train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

In [23]:
# Cargar el DataFrame de pandas en un objeto tf.data.Dataset
# armo según lo que me interesa
train_dataset = tf.data.Dataset.from_tensor_slices((X_train.values, 
                                              y_train))



validation_dataset = tf.data.Dataset.from_tensor_slices((X_test.values, 
                                              y_test))

In [24]:
# veo un dato de train
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

text:  b'@ menyrbie @ phil_gahan @ chrisitv http : //t.co/ifz9fan2pa http : //t.co/xx6ghgfzcc http : //t.co/i2nlzdxno8'
label:  [0. 1. 0.]


In [25]:
# veo un dato de train
for example, label in train_dataset.take(2):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

text:  b'@ menyrbie @ phil_gahan @ chrisitv http : //t.co/ifz9fan2pa http : //t.co/xx6ghgfzcc http : //t.co/i2nlzdxno8'
label:  [0. 1. 0.]
text:  b'advice talk neighbour family exchange phone number create contact list phone number neighbour school employer chemist gp set online shopping account po adequate supply regular med order'
label:  [0. 0. 1.]


In [26]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [27]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
validation_dataset = validation_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [28]:
for example, label in train_dataset.take(1):
  print(f'texts:  {example.numpy()[:3]}\n')
  print(f'labels: , {label.numpy()[:3]}')

texts:  [b'economic impact low income family taking toll issue shared today driver small company laid work w 4 mouth feed grocery price rising one watch kid need work employer pay'
 b'u complaining school shut u supermarket nh staff come work hardly choice basically get verbally abused stock best'
 b'@ menyrbie @ phil_gahan @ chrisitv http : //t.co/ifz9fan2pa http : //t.co/xx6ghgfzcc http : //t.co/i2nlzdxno8']

labels: , [[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


<a name="4.8"></a>
### <b> <font color='#5499C7'> 4.8. Text Vectorization layer </font> </b>

In [29]:
VOCAB_SIZE = 10000
max_length = 45 # max length our sequences will be (e.g. how many words from a Tweet does a model see?)


encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE,
                                    output_mode="int")
                                    #output_sequence_length=max_length)

# Fit the text vectorizer instance to the training data using the adapt() method
encoder.adapt(train_dataset.map(lambda text, label: text))


In [30]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'http', 'coronavirus', 'covid19', 'price', 'store',
       'supermarket', 'food', 'grocery', 'people', 'amp', 'consumer',
       '19', 'covid', 'shopping', 's', 'online', 'need', 'pandemic'],
      dtype='<U27')

In [31]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[ 223,   94,  138,  672,  107,  192, 2037,  327, 1787,   62,  209,
         243,   92, 1459,   38,  461,  197, 1476,  860,    9,    5,  772,
          34,  419,  396,   18,   38, 1850,  183,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0],
       [  23, 3041,  390,  540,   23,    7,  301,   81,  147,   38, 3716,
        1143, 2099,   21, 5616, 4312,   33,  224,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0],
       [   1,    1,    1,    2,    1,    2,    1,    2,    1,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0]])

In [32]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

Original:  b'economic impact low income family taking toll issue shared today driver small company laid work w 4 mouth feed grocery price rising one watch kid need work employer pay'
Round-trip:  economic impact low income family taking toll issue shared today driver small company laid work w 4 mouth feed grocery price rising one watch kid need work employer pay          

Original:  b'u complaining school shut u supermarket nh staff come work hardly choice basically get verbally abused stock best'
Round-trip:  u complaining school shut u supermarket nh staff come work hardly choice basically get verbally abused stock best                     

Original:  b'@ menyrbie @ phil_gahan @ chrisitv http : //t.co/ifz9fan2pa http : //t.co/xx6ghgfzcc http : //t.co/i2nlzdxno8'
Round-trip:  [UNK] [UNK] [UNK] http [UNK] http [UNK] http [UNK]                              



In [33]:
embedding = tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, # set the input shape; size of our vocabulary
                                 output_dim=128, # set the size of the embedding vector
                                 embeddings_initializer="uniform", # default, initialize embedding vectors randomly
                                 input_length=max_length # how long is each input
                             )

embedding

In [ ]:
from tensorflow.keras import layers,callbacks,models,Sequential,losses

INPUT_SHAPE=(1,)

def build_model_1(input_shape,name):
    inputs = layers.Input(shape=input_shape, dtype=tf.string) # inputs are 1-dimensional strings
    x = encoder(inputs) # turn the input text into numbers 
    x = embedding(x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    outputs = tf.keras.layers.Dense(3, activation="softmax")(x) # dsp.  num_classes
    model = tf.keras.Model(inputs, outputs, name=name) # construct the model
    return model


model_1 = build_model_1(INPUT_SHAPE,'model_1')


# Compile model
model_1.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy', 'Precision','Recall'])

# Fit the model
history_1 = model_1.fit(train_dataset,
                        epochs=5,
                        verbose=0,
                        validation_data=validation_dataset)

In [ ]:
# evaluate
score1 = model_1.evaluate(validation_dataset)